In [1]:
!python -V

Python 3.9.12


# Packages

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import mlflow #package for python
from mlflow.data.pandas_dataset import PandasDataset

# Data Processing Stage

In [2]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        
        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename, engine = 'pyarrow')

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
# Read data

df_train = read_dataframe('/home/Pranoy/MLOPS-Practice/data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('/home/Pranoy/MLOPS-Practice/data/yellow_tripdata_2023-02.parquet')

In [4]:
df_train.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [5]:
len(df_train), len(df_val)

(3009173, 2855951)

In [6]:
df_train = df_train.sample(frac=0.20)
df_train.reset_index(inplace=True)

df_val = df_val.sample(frac=0.20)
df_val.reset_index(inplace=True)

len(df_train), len(df_val)

(601835, 571190)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# ML Flow Stage

Initialize mlflow (i.e., hosting the mlflow in our VM instance) using the below command:

 - mlflow ui --backend-store-uri sqlite:///mlflow.db

Once initalize then establish the connection between MLflow server with your script using below command:

 - mlflow.set_tracking_uri("sqlite:///mlflow.db")
 - mlflow.set_experiment("nyc-taxi-experiment")



In [10]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("practice-nyc-taxi-experiment")

<Experiment: artifact_location='/home/Pranoy/MLOPS-Practice/week1/mlruns/1', creation_time=1702062308606, experiment_id='1', last_update_time=1702062308606, lifecycle_stage='active', name='practice-nyc-taxi-experiment', tags={}>

## Regression Flow

In [19]:
mlflow.start_run(run_name="pranoy_test_LR1", description="My first Linear Regression logged")
mlflow.log_param("model","LinearRegression")
mlflow.set_tag("Name","Pranoy Dewanjee")
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)
print(mse)
mlflow.end_run()


5.308733084398722


In [20]:
mlflow.start_run(run_name="pranoy_test_LR2", description="My second Linear Regression logged")
mlflow.log_param("model","LinearRegression")
mlflow.set_tag("Name","Pranoy Dewanjee")
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)
print(mse)
mlflow.end_run()


5.308733084398722


In [21]:
# Updating the existing experiment by passing run_id in start_run()
mlflow.start_run(run_id="cbe118eef2414930952f7f56e5b4d234")

mlflow.log_param("status","checking")

mlflow.end_run()


In [18]:
# delete an experiment permanently
mlflow.delete_experiment(experiment_id="2")

In [26]:
alpha = 0.1
mlflow.start_run(run_name="pranoy_test_Lasso1",description="My first Lasso Regression Logged")
mlflow.log_param("alpha", alpha)
mlflow.log_param("model","Lasso")
mlflow.set_tags({'FirstName':'Pranoy'
                 ,'LastName':'Dewanjee'})
ls = Lasso(alpha)
ls.fit(X_train, y_train)

y_pred = ls.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(rmse)
mlflow.end_run()

10.08645685623584


In [27]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [28]:
with open('models/lasso_reg.bin', 'wb') as f_out:
    pickle.dump((dv, ls), f_out)

In [13]:
with mlflow.start_run(run_name="Pranoy_test_Lasso2", description="My Second Lasso Regression Logged"):

    mlflow.set_tag("developer", "Pranoy")

    mlflow.log_param("train-data-path", "/home/Pranoy/MLOPS-Practice/data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "/home/Pranoy/MLOPS-Practice/data/yellow_tripdata_2023-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lasso_reg.bin", artifact_path="models_pickle")
    mlflow.log_artifacts(local_dir="/home/Pranoy/MLOPS-Practice/data",artifact_path="dataset")

    train_dataset: PandasDataset = mlflow.data.from_pandas(df_train)
    validation_dataset: PandasDataset = mlflow.data.from_pandas(df_val)
    
    mlflow.log_input(train_dataset, context="Training", tags={"description":"yellow taxi 2023-01"})
    mlflow.log_input(validation_dataset, context="Validation", tags={"description":"yellow taxi 2023-02"})
    

/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/mlflow/data/digest_utils.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/mlflow/data/digest_utils.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
2023/12/08 16:11:21 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Unable to map 'object' type to MLflow DataType. object can be mapped iff all values have identical data type which is one of (string, (bytes or byterray),  int, float).
2023/12/08 16:11:21 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Unable to map 'object' type to MLflow DataType. object can be mapped iff al

## XG Boost Flow


In [11]:
# Packages
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
count = 0
def objective(params):
    global count
    count+=1
    with mlflow.start_run(run_name=f'Pranoy_XGBoost_{count}'):
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=5,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:37:51] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.46949                          
[1]	validation-rmse:8.96500                          
[2]	validation-rmse:8.51030                          
[3]	validation-rmse:8.10117                          
[4]	validation-rmse:7.73390                          
 33%|███▎      | 1/3 [00:13<00:27, 13.63s/trial, best loss: 7.733900964172708]

/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:38:04] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.65755                                                   
[1]	validation-rmse:5.42472                                                   
[2]	validation-rmse:5.02400                                                   
[3]	validation-rmse:4.88508                                                   
[4]	validation-rmse:4.83294                                                   
 67%|██████▋   | 2/3 [00:43<00:23, 23.09s/trial, best loss: 4.832936335906277]

/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:38:34] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.53844                                                   
[1]	validation-rmse:7.44107                                                   
[2]	validation-rmse:6.64897                                                   
[3]	validation-rmse:6.08883                                                   
[4]	validation-rmse:5.69958                                                   
100%|██████████| 3/3 [01:09<00:00, 23.16s/trial, best loss: 4.832936335906277]


In [14]:
best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=5,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/12/08 19:35:12 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2023/12/08 19:35:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fdaab9cba27c4b68bdc89b9733d90fa9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


: 

In [14]:
mlflow.xgboost.autolog(disable=True)

In [16]:
with mlflow.start_run(run_name="Pranoy_XGBoost_4"):
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=5,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:39:32] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.33338
[1]	validation-rmse:8.72336
[2]	validation-rmse:8.18889
[3]	validation-rmse:7.72291
[4]	validation-rmse:7.31764


/home/Pranoy/.local/share/virtualenvs/week1-y_khLokL/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:39:50] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


##